In [4]:
import os
import shutil

# Ruta donde se encuentran todos los archivos
source_dir = './Dataset_500_EMPRESAS'  # Cambia esto si es necesario

# Obtener todos los archivos en el directorio
files = os.listdir(source_dir)

# Crear un conjunto de todos los tickers únicos
tickers = set()
for file in files:
    if file.endswith('.csv') or file.endswith('.json'):
        parts = file.split('_')
        if len(parts) >= 2:
            ticker = parts[0]
            tickers.add(ticker)

# Crear carpetas y mover archivos
for ticker in tickers:
    folder_path = os.path.join(source_dir, ticker)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    for file in files:
        if file.startswith(ticker + '_'):
            source_file = os.path.join(source_dir, file)
            dest_file = os.path.join(folder_path, file)
            shutil.move(source_file, dest_file)
            print(f"Moved {file} to {folder_path}")

print("Organización completada.")

Moved NFLX_cash_flow.csv to ./Dataset_500_EMPRESAS/NFLX
Moved NFLX_balance_sheet.csv to ./Dataset_500_EMPRESAS/NFLX
Moved NFLX_info.json to ./Dataset_500_EMPRESAS/NFLX
Moved NFLX_income_statement.csv to ./Dataset_500_EMPRESAS/NFLX
Moved NFLX_historico.csv to ./Dataset_500_EMPRESAS/NFLX
Moved GOOGL_cash_flow.csv to ./Dataset_500_EMPRESAS/GOOGL
Moved GOOGL_income_statement.csv to ./Dataset_500_EMPRESAS/GOOGL
Moved GOOGL_info.json to ./Dataset_500_EMPRESAS/GOOGL
Moved GOOGL_balance_sheet.csv to ./Dataset_500_EMPRESAS/GOOGL
Moved GOOGL_historico.csv to ./Dataset_500_EMPRESAS/GOOGL
Moved DIS_historico.csv to ./Dataset_500_EMPRESAS/DIS
Moved DIS_cash_flow.csv to ./Dataset_500_EMPRESAS/DIS
Moved DIS_income_statement.csv to ./Dataset_500_EMPRESAS/DIS
Moved DIS_info.json to ./Dataset_500_EMPRESAS/DIS
Moved DIS_balance_sheet.csv to ./Dataset_500_EMPRESAS/DIS
Moved IQV_cash_flow.csv to ./Dataset_500_EMPRESAS/IQV
Moved IQV_balance_sheet.csv to ./Dataset_500_EMPRESAS/IQV
Moved IQV_info.json to ./Da

In [6]:
import pandas as pd
import os
import json

def procesar_ticker(ticker, base_path):
    # Cargar datos históricos
    hist_path = os.path.join(base_path, ticker, f"{ticker}_historico.csv")
    df_hist = pd.read_csv(hist_path, parse_dates=['Date'], index_col='Date')
    
    # Cargar datos financieros
    financial_dfs = []
    for tipo in ['cash_flow', 'balance_sheet', 'income_statement']:
        path = os.path.join(base_path, ticker, f"{ticker}_{tipo}.csv")
        if not os.path.exists(path):
            print(f"Advertencia: No existe {path}")
            continue
        df = pd.read_csv(path, index_col=0)
        df = df.T  # Transponer para que las fechas sean el índice
        df.index = pd.to_datetime(df.index)
        financial_dfs.append(df)
    
    if not financial_dfs:
        print(f"No hay datos financieros para {ticker}")
        return
    
    # Combinar datos financieros
    df_financial = pd.concat(financial_dfs, axis=1)
    
    # Crear índice diario covering el rango de fechas históricas
    start_date = df_hist.index.min()
    end_date = df_hist.index.max()
    idx_diario = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Reindexar datos financieros al índice diario
    df_financial_diario = df_financial.reindex(idx_diario)
    
    # Forward fill para propagar valores
    df_financial_diario = df_financial_diario.ffill()
    
    # Rellenar con 0 las fechas anteriores al primer reporte financiero
    if not df_financial.empty:
        first_financial_date = df_financial.index.min()
        mask = idx_diario < first_financial_date
        df_financial_diario.loc[mask] = 0
    
    # Combinar con datos históricos
    df_combinado = df_hist.join(df_financial_diario, how='left')
    
    # Cargar sectorKey desde info.json
    info_path = os.path.join(base_path, ticker, f"{ticker}_info.json")
    with open(info_path, 'r') as f:
        info_data = json.load(f)
    sector_key = info_data.get('sectorKey', '')
    
    # Añadir sectorKey como columna
    df_combinado['sectorKey'] = sector_key
    
    # Guardar resultado
    output_path = os.path.join(base_path, f"{ticker}_completo.csv")
    df_combinado.to_csv(output_path, index_label='Date')
    print(f"Guardado: {output_path}")

# Procesar todos los tickers
base_path = './Dataset_500_EMPRESAS'  # Cambia esto si es necesario
tickers = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

for ticker in tickers:
    try:
        procesar_ticker(ticker, base_path)
    except Exception as e:
        print(f"Error procesando {ticker}: {str(e)}")

Guardado: ./Dataset_500_EMPRESAS/CTAS_completo.csv
Guardado: ./Dataset_500_EMPRESAS/WELL_completo.csv
Guardado: ./Dataset_500_EMPRESAS/VZ_completo.csv
Guardado: ./Dataset_500_EMPRESAS/AMZN_completo.csv
Guardado: ./Dataset_500_EMPRESAS/CNP_completo.csv
Guardado: ./Dataset_500_EMPRESAS/RCL_completo.csv
Guardado: ./Dataset_500_EMPRESAS/CAT_completo.csv
Guardado: ./Dataset_500_EMPRESAS/TFC_completo.csv
Guardado: ./Dataset_500_EMPRESAS/AAPL_completo.csv
Guardado: ./Dataset_500_EMPRESAS/BF-B_completo.csv
Guardado: ./Dataset_500_EMPRESAS/PANW_completo.csv
Guardado: ./Dataset_500_EMPRESAS/PM_completo.csv
Guardado: ./Dataset_500_EMPRESAS/KHC_completo.csv
Guardado: ./Dataset_500_EMPRESAS/TEL_completo.csv
Guardado: ./Dataset_500_EMPRESAS/CMCSA_completo.csv
Guardado: ./Dataset_500_EMPRESAS/GRMN_completo.csv
Guardado: ./Dataset_500_EMPRESAS/ANET_completo.csv
Guardado: ./Dataset_500_EMPRESAS/BRO_completo.csv
Guardado: ./Dataset_500_EMPRESAS/CAH_completo.csv
Guardado: ./Dataset_500_EMPRESAS/SW_comple

In [7]:
import pandas as pd
import os
import numpy as np

def limpiar_datos(ticker, input_dir, output_dir):
    # Leer el archivo CSV
    input_path = os.path.join(input_dir, f"{ticker}_completo.csv")
    df = pd.read_csv(input_path)
    
    # Convertir la columna 'Free Cash Flow' a numérico, manejando errores
    df['Free Cash Flow'] = pd.to_numeric(df['Free Cash Flow'], errors='coerce')
    
    # Filtrar filas donde 'Free Cash Flow' no es NaN, vacío ni 0
    df_limpio = df[df['Free Cash Flow'].notna() & (df['Free Cash Flow'] != 0)]
    
    # Guardar el resultado
    output_path = os.path.join(output_dir, f"{ticker}_completo_arreglado.csv")
    df_limpio.to_csv(output_path, index=False)
    print(f"Archivo limpio guardado: {output_path}")

# Directorios de entrada y salida
input_dir = './Dataset_500_completo'  # Cambia esto a la ruta de tus archivos completos
output_dir = './datos_limpios'  # Carpeta para los archivos limpios

# Crear directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Obtener todos los archivos completos
archivos = [f for f in os.listdir(input_dir) if f.endswith('_completo.csv')]
tickers = [f.split('_')[0] for f in archivos]

# Procesar cada ticker
for ticker in tickers:
    try:
        limpiar_datos(ticker, input_dir, output_dir)
    except Exception as e:
        print(f"Error procesando {ticker}: {str(e)}")

print("Proceso completado. Archivos limpios guardados en:", output_dir)

Archivo limpio guardado: ./datos_limpios/NWSA_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/CF_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/TSCO_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/AJG_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/GEN_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/LOW_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/HIG_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/LYV_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/XOM_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/TEL_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/PODD_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/XYL_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/EMR_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/SBUX_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/PEG_

In [8]:
import pandas as pd
import os
import numpy as np

def encontrar_columnas_comunes(directorio, archivo_referencia):
    """
    Encuentra las columnas comunes entre todos los archivos CSV en el directorio
    basándose en un archivo de referencia.
    """
    # Leer las columnas del archivo de referencia
    referencia = pd.read_csv(os.path.join(directorio, archivo_referencia), nrows=0)
    columnas_comunes = set(referencia.columns)
    
    # Listar todos los archivos CSV en el directorio
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    
    # Encontrar la intersección de columnas entre todos los archivos
    for archivo in archivos:
        if archivo == archivo_referencia:
            continue
            
        try:
            df_temp = pd.read_csv(os.path.join(directorio, archivo), nrows=0)
            columnas_comunes = columnas_comunes.intersection(set(df_temp.columns))
        except Exception as e:
            print(f"Error leyendo {archivo}: {str(e)}")
            continue
    
    print(f"Se encontraron {len(columnas_comunes)} columnas comunes")
    return list(columnas_comunes)

In [9]:
def normalizar_dataset(directorio, columnas_comunes, directorio_salida=None):
    """
    Normaliza todos los archivos CSV en el directorio manteniendo solo las columnas comunes.
    """
    if directorio_salida is None:
        directorio_salida = directorio
    
    # Crear directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
    
    # Listar todos los archivos CSV en el directorio
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    
    for archivo in archivos:
        try:
            # Leer el archivo
            df = pd.read_csv(os.path.join(directorio, archivo))
            
            # Mantener solo las columnas comunes
            columnas_existentes = [col for col in columnas_comunes if col in df.columns]
            df_normalizado = df[columnas_existentes]
            
            # Guardar el archivo normalizado
            ruta_salida = os.path.join(directorio_salida, archivo)
            df_normalizado.to_csv(ruta_salida, index=False)
            
            print(f"Archivo {archivo} normalizado. Se mantuvieron {len(columnas_existentes)} columnas.")
            
        except Exception as e:
            print(f"Error procesando {archivo}: {str(e)}")
            continue


In [10]:

# Configuración
directorio = "./datos_limpios"  # Cambia esto por tu directorio
archivo_referencia = "BX_completo_arreglado.csv"  # Archivo con las 111 columnas
directorio_salida = "./datos_normalizados"  # Directorio para archivos normalizados

# Encontrar columnas comunes
columnas_comunes = encontrar_columnas_comunes(directorio, archivo_referencia)

# Normalizar dataset
normalizar_dataset(directorio, columnas_comunes, directorio_salida)

# Verificar que todos los archivos tengan las mismas columnas
def verificar_consistencia(directorio):
    """
    Verifica que todos los archivos en el directorio tengan las mismas columnas.
    """
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    columnas_base = None
    
    for archivo in archivos:
        df_temp = pd.read_csv(os.path.join(directorio, archivo), nrows=0)
        
        if columnas_base is None:
            columnas_base = set(df_temp.columns)
            print(f"Columnas base ({len(columnas_base)}): {sorted(columnas_base)}")
        else:
            columnas_actual = set(df_temp.columns)
            if columnas_base != columnas_actual:
                print(f"Advertencia: {archivo} tiene columnas diferentes")
                print(f"Faltantes: {columnas_base - columnas_actual}")
                print(f"Adicionales: {columnas_actual - columnas_base}")
    
    print("Verificación de consistencia completada")

# Verificar consistencia
verificar_consistencia(directorio_salida)

Se encontraron 48 columnas comunes
Archivo GOOGL_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo SRE_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo STT_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo PGR_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo LEN_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo TDY_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo VRSN_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo IVZ_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo MTCH_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo SYK_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo SW_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo NXPI_completo_arreglado.csv normalizado. Se mantuvieron 48 columnas.
Archivo RTX_completo_arreglado.csv normalizado. S

In [3]:
 !pip install TA-Lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.9/999.9 kB 15.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [TA-Lib]2m3/4 [TA-Lib]


In [1]:
import pandas as pd
import glob

def obtener_sectores_unicos(path_pattern="data/*.csv", col_sector="sectorKey"):
    """
    Lee múltiples CSV y devuelve una lista de todos los sectores únicos.
    
    Args:
        path_pattern (str): patrón de búsqueda para archivos CSV (ej: 'data/*.csv')
        col_sector (str): nombre de la columna que contiene el sector
    
    Returns:
        list: lista de sectores únicos encontrados en todos los archivos
    """
    sectores = set()
    
    # Buscar todos los CSV en la ruta indicada
    for file in glob.glob(path_pattern):
        df = pd.read_csv(file)
        if col_sector in df.columns:
            sectores.update(df[col_sector].dropna().unique())
    
    return sorted(list(sectores))

# Ejemplo de uso
sectores_totales = obtener_sectores_unicos("./datos_normalizados/*.csv", col_sector="sectorKey")
print("Sectores encontrados:", sectores_totales)

Sectores encontrados: ['basic-materials', 'communication-services', 'consumer-cyclical', 'consumer-defensive', 'energy', 'financial-services', 'healthcare', 'industrials', 'real-estate', 'technology', 'utilities']


In [2]:
len(sectores_totales)

11

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import talib as ta


def calculate_technical_indicators(df):
    """Calcula 48 indicadores técnicos y agrega datos financieros"""
    # Precios
    open_price = df['Open'].values
    high = df['High'].values
    low = df['Low'].values
    close = df['Close'].values
    volume = df['Volume'].values
    
    features = {}
    
    # ===== INDICADORES TÉCNICOS (48 features) =====
    
    # Medias móviles (8 features)
    features['SMA_5'] = ta.SMA(close, timeperiod=5)
    features['SMA_10'] = ta.SMA(close, timeperiod=10)
    features['SMA_20'] = ta.SMA(close, timeperiod=20)
    features['SMA_50'] = ta.SMA(close, timeperiod=50)
    features['EMA_5'] = ta.EMA(close, timeperiod=5)
    features['EMA_10'] = ta.EMA(close, timeperiod=10)
    features['EMA_20'] = ta.EMA(close, timeperiod=20)
    features['EMA_50'] = ta.EMA(close, timeperiod=50)
    
    # Osciladores (12 features)
    features['RSI'] = ta.RSI(close)
    features['STOCH_K'], features['STOCH_D'] = ta.STOCH(high, low, close)
    features['MACD'], features['MACD_Signal'], features['MACD_Hist'] = ta.MACD(close)
    features['ADX'] = ta.ADX(high, low, close)
    features['CCI'] = ta.CCI(high, low, close)
    features['WILLR'] = ta.WILLR(high, low, close)
    features['ULTOSC'] = ta.ULTOSC(high, low, close)
    features['ROC'] = ta.ROC(close)
    
    # Volatilidad (6 features)
    features['ATR'] = ta.ATR(high, low, close)
    features['NATR'] = ta.NATR(high, low, close)
    features['TRANGE'] = ta.TRANGE(high, low, close)
    features['BB_Upper'], features['BB_Middle'], features['BB_Lower'] = ta.BBANDS(close)
    
    # Volumen (5 features)
    # features['OBV'] = ta.OBV(close, volume)
    # features['AD'] = ta.AD(high, low, close, volume)
    # features['ADOSC'] = ta.ADOSC(high, low, close, volume)
    # features['MFI'] = ta.MFI(high, low, close, volume)
    
    # Momentum (8 features)
    features['MOM'] = ta.MOM(close)
    features['PPO'] = ta.PPO(close)
    features['APO'] = ta.APO(close)
    features['AROON_Up'], features['AROON_Down'] = ta.AROON(high, low)
    features['AROONOSC'] = ta.AROONOSC(high, low)
    features['BOP'] = ta.BOP(open_price, high, low, close)
    
    # Precios relativos (9 features)
    features['Price_Change'] = close / np.roll(close, 1) - 1
    features['High_Low_Ratio'] = high / low
    features['Close_Open_Ratio'] = close / open_price
    # features['Volume_Change'] = volume / np.roll(volume, 1) - 1
    
    # ===== DATOS FINANCIeros DEL CSV (46 features adicionales) =====
    
    # Flujo de caja y liquidez
    features['Free_Cash_Flow'] = df['Free Cash Flow']
    features['Operating_Cash_Flow'] = df['Operating Cash Flow']
    features['Investing_Cash_Flow'] = df['Investing Cash Flow']
    features['Financing_Cash_Flow'] = df['Financing Cash Flow']
    features['Beginning_Cash_Position'] = df['Beginning Cash Position']
    features['End_Cash_Position'] = df['End Cash Position']
    features['Changes_In_Cash'] = df['Changes In Cash']
    features['Cash_And_Cash_Equivalents'] = df['Cash And Cash Equivalents']
    
    # Rentabilidad
    features['Net_Income'] = df['Net Income']
    features['Pretax_Income'] = df['Pretax Income']
    features['Total_Revenue'] = df['Total Revenue']
    features['Operating_Revenue'] = df['Operating Revenue']
    features['Normalized_Income'] = df['Normalized Income']
    features['Net_Income_Common_Stockholders'] = df['Net Income Common Stockholders']
    features['Net_Income_From_Continuing_Operations'] = df['Net Income From Continuing Operations']
    features['Net_Income_From_Continuing_And_Discontinued_Operation'] = df['Net Income From Continuing And Discontinued Operation']
    features['Net_Income_Including_Noncontrolling_Interests'] = df['Net Income Including Noncontrolling Interests']
    features['Net_Income_Continuous_Operations'] = df['Net Income Continuous Operations']
    features['Diluted_NI_Availto_Com_Stockholders'] = df['Diluted NI Availto Com Stockholders']
    features['Net_Income_From_Continuing_Operation_Net_Minority_Interest'] = df['Net Income From Continuing Operation Net Minority Interest']
    
    # Por acción
    features['Basic_EPS'] = df['Basic EPS']
    features['Diluted_EPS'] = df['Diluted EPS']
    
    # Acciones
    features['Diluted_Average_Shares'] = df['Diluted Average Shares']
    features['Basic_Average_Shares'] = df['Basic Average Shares']
    features['Share_Issued'] = df['Share Issued']
    features['Ordinary_Shares_Number'] = df['Ordinary Shares Number']
    features['Capital_Stock'] = df['Capital Stock']
    features['Common_Stock'] = df['Common Stock']
    
    # Balance general
    features['Total_Assets'] = df['Total Assets']
    features['Total_Liabilities_Net_Minority_Interest'] = df['Total Liabilities Net Minority Interest']
    features['Total_Equity_Gross_Minority_Interest'] = df['Total Equity Gross Minority Interest']
    features['Common_Stock_Equity'] = df['Common Stock Equity']
    features['Stockholders_Equity'] = df['Stockholders Equity']
    features['Total_Capitalization'] = df['Total Capitalization']
    features['Invested_Capital'] = df['Invested Capital']
    features['Net_Tangible_Assets'] = df['Net Tangible Assets']
    features['Tangible_Book_Value'] = df['Tangible Book Value']
    
    # Deuda
    features['Total_Debt'] = df['Total Debt']
    features['Long_Term_Debt_And_Capital_Lease_Obligation'] = df['Long Term Debt And Capital Lease Obligation']
    
    # Ratios financieros
    features['Tax_Rate_For_Calcs'] = df['Tax Rate For Calcs']
    features['Tax_Effect_Of_Unusual_Items'] = df['Tax Effect Of Unusual Items']
    features['Close'] = df['Close']
    features['Date'] = df['Date']
    
    # Sector (convertir a numérico si es necesario)
    # if 'sectorKey' in df.columns:
    #     # One-hot encoding para sector
    #     sector_dummies = pd.get_dummies(df['sectorKey'], prefix='Sector')
    #     for col in sector_dummies.columns:
    #         features[col] = sector_dummies[col]
    if 'sectorKey' in df.columns:
        # Generar one-hot encoding SOLO de los sectores presentes en este df
        sector_dummies = pd.get_dummies(df['sectorKey'], prefix='Sector')

        # Crear todas las columnas esperadas en base a sectores_totales
        expected_cols = [f"Sector_{s}" for s in sectores_totales]

        # Reindexar para incluir todos los sectores, rellenando con 0 donde falten
        sector_dummies = sector_dummies.reindex(columns=expected_cols, fill_value=0)

        # Convertir booleanos a enteros (0 o 1)
        sector_dummies = sector_dummies.astype(int)

        # Agregar cada columna dummy al diccionario de features
        for col in sector_dummies.columns:
            features[col] = sector_dummies[col]  
    
    # Convertir a DataFrame
    features_df = pd.DataFrame(features, index=df.index)
    
    # Eliminar columnas que puedan tener todos valores NaN
    features_df = features_df.dropna(axis=1, how='all')
    
    # Llenar valores NaN restantes con 0 o método apropiado
    features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    return features_df


In [8]:
def create_target_variable(df, horizon):
    """Crea la variable objetivo: % cambio en los próximos 3 meses (63 días trading)"""
    # Asegurarse de que los datos están ordenados por fecha
    df = df.sort_index()
    
    # Calcular precio futuro (3 meses después)
    future_prices = df['Close'].shift(-horizon)
    current_prices = df['Close']
    
    # Calcular porcentaje de cambio
    target = ((future_prices - current_prices) / current_prices) * 100
    
    # Crear DataFrame con target
    target_df = pd.DataFrame({
        'Target_3M_Return': target,
        'Target_Direction': np.where(target > 0, 1, 0)  # 1: subida, 0: bajada
    }, index=df.index)
    df_with_target = df.join(target_df, how='left')
    
    # Eliminar valores donde no hay futuro disponible
    df_with_target = df_with_target.dropna()
    
    return df_with_target

In [9]:
def data_augmentation_dataset(directorio, directorio_salida=None):
    """
    Normaliza todos los archivos CSV en el directorio manteniendo solo las columnas comunes.
    """
    if directorio_salida is None:
        directorio_salida = directorio
    
    # Crear directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
    
    # Listar todos los archivos CSV en el directorio
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    # archivos = ["A_completo_arreglado.csv"]
    
    for archivo in archivos:
        try:
            # Leer el archivo
            df = pd.read_csv(os.path.join(directorio, archivo))
            df_2 = calculate_technical_indicators(df=df)
            df_3 = create_target_variable(df=df_2, horizon= 60)

            # Guardar el archivo normalizado
            ruta_salida = os.path.join(directorio_salida, archivo)
            df_3.to_csv(ruta_salida, index=False)
            
        except Exception as e:
            print(f"Error procesando {archivo}: {str(e)}")
            continue


In [10]:
import os
import shutil

# Configuración
directorio = "./datos_normalizados"  # Cambia esto por tu directorio
directorio_salida = "./data_augmentation"  # Directorio para archivos normalizados

# Encontrar columnas comunes
data_augmentation_dataset(directorio, directorio_salida)

/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: Futu

In [11]:
def create_target_variable_v2(df, horizon):
    """Crea la variable objetivo: % cambio en los próximos 3 meses (63 días trading)"""
    # Asegurarse de que los datos están ordenados por fecha
    df = df.sort_index()
    
    # Calcular precio futuro (3 meses después)
    future_prices = df['Close'].shift(-horizon)
    current_prices = df['Close']
    
    # Calcular porcentaje de cambio
    target = ((future_prices - current_prices) / current_prices) * 100
    
    # Crear DataFrame con target
    target_df = pd.DataFrame({
        'Target_3M_Return': target,
        'Target_Direction': np.where(target > 0, 1, 0)  # 1: subida, 0: bajada
    }, index=df.index)
    df_with_target = df.join(target_df, how='left')
    
    # Eliminar valores donde no hay futuro disponible
    # df_with_target = df_with_target.dropna()
    
    return df_with_target

In [12]:
def data_augmentation_dataset_with_unknow_values(directorio, directorio_salida=None):
    """
    Normaliza todos los archivos CSV en el directorio manteniendo solo las columnas comunes.
    """
    if directorio_salida is None:
        directorio_salida = directorio
    
    # Crear directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
    
    # Listar todos los archivos CSV en el directorio
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    # archivos = ["A_completo_arreglado.csv"]
    
    for archivo in archivos:
        try:
            # Leer el archivo
            df = pd.read_csv(os.path.join(directorio, archivo))
            df_2 = calculate_technical_indicators(df=df)
            df_3 = create_target_variable_v2(df=df_2, horizon= 60)

            # Guardar el archivo normalizado
            ruta_salida = os.path.join(directorio_salida, archivo)
            df_3.to_csv(ruta_salida, index=False)
            
        except Exception as e:
            print(f"Error procesando {archivo}: {str(e)}")
            continue

In [13]:
import os
import shutil

# Configuración
directorio = "./datos_normalizados"  # Cambia esto por tu directorio
directorio_salida = "./data_augmentation_unknow_values"  # Directorio para archivos normalizados

# Encontrar columnas comunes
data_augmentation_dataset_with_unknow_values(directorio, directorio_salida)

/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_48398/1441644667.py:157: Futu